In [1]:
import pandas as pd
import simplejson

In [2]:
base_path = "../data/"

In [3]:
book_file = "book-A-song-of-Ice-and-Fire.csv"

In [4]:
tv_file = "tv-Game-of-Thrones.csv"

In [5]:
bookdf = pd.read_csv(base_path + book_file)
bookdf["category"] = "book"

In [6]:
tvdf = pd.read_csv(base_path + tv_file)
tvdf['category'] = "tv"

In [7]:
fb  = open("../data/book_characters.json", "r")
book_char_list = simplejson.loads(fb.read())

In [8]:
ft  = open("../data/tv_characters.json", "r")
tv_char_list = simplejson.loads(ft.read())

In [9]:
#One character as listed under two names. 
#Dropping this column because it had fewer enteires, all of which were duplicate to the surviving columnn. 
bookdf = bookdf.drop('Aegon T./Egg', 1)
book_char_list.remove('Aegon T./Egg')

In [10]:
def rename_longer(cb, ct):
    global tv_char_list
    global book_char_list
    if len(cb) > len(ct) and "Nymeria" not in cb:
        tvdf.rename(columns={ct: cb}, inplace=True)
        tv_char_list = [cb if x==ct else x for x in tv_char_list]
        print "Book: {}, TV: {} changed in tv".format(cb, ct)
    elif "Nymeria" not in cb:
        bookdf.rename(columns={cb: ct}, inplace=True)
        book_char_list = [ct if x==cb else x for x in book_char_list]
        print "Book: {}, TV: {} changed in book".format(cb, ct)
        
for cb in book_char_list:
    if cb not in tv_char_list:
        for ct in tv_char_list:
                if ct.split("/")[0] == cb:
                    rename_longer(cb, ct)
                elif cb.split("/")[0] == ct:
                    rename_longer(cb, ct)
                elif cb in ct:
                    rename_longer(cb, ct)
                elif ct in cb:
                    rename_longer(cb, ct)

rename_longer("Catelyn T.", "Catelyn S.")
rename_longer("Brienne T.", "Brienne of Tarth")

Book: Aemon T./The Dragonknight, TV: Aemon T. changed in tv
Book: Brynden T., TV: Brynden T./Brynden the Blackfish changed in book
Book: Daenerys T./Dany, TV: Daenerys T. changed in tv
Book: Eddard S./Ned, TV: Eddard S. changed in tv
Book: Gendry, TV: Gendry W. changed in book
Book: Myranda R., TV: Myranda changed in tv
Book: Petyr B./Littlefinger, TV: Petyr B. changed in tv
Book: Rhaenys T./Queen Who Never Was, TV: Rhaenys T. changed in tv
Book: Shaera T., TV: Shae changed in tv
Book: Catelyn T., TV: Catelyn S. changed in book
Book: Brienne T., TV: Brienne of Tarth changed in book


In [11]:
for column in list(bookdf):
    if list(bookdf).count(column) >1:
        print column, bookdf[column].sum()

In [18]:
#merge tables using TV as the standard since it has less characters and only characters appearing in both will be popular enough to appear on the chart. 
#check if headers only in tv are important people
tv_not_book = [x for x in list(tvdf) if x not in list(bookdf)]
for i in tv_not_book:
    print "{} : tv-{}".format(i, tvdf[i].sum())

Direwolves : tv-38
Dragons : tv-12
G. M. Pycelle : tv-3
Grey Worm : tv-10
Karl T. : tv-1
Khal Drogo : tv-91
Locke : tv-2
Ludd W. : tv-1
M. Luwin : tv-5
Rodrick F. : tv-1
Royland D. : tv-1
Ryon F. : tv-1
S. Mordane : tv-1
Talia F. : tv-1
Talisa M. : tv-21
Viserion T. : tv-1
Yara G. : tv-28
the three eyed raven : tv-1


In [ ]:
#innerjoing the tables. Will result in droping above characters from tv set. 

In [27]:
gotdf = pd.concat([bookdf,tvdf], join="inner")
got_char_list = [x for x in tv_char_list if x in book_char_list]

In [26]:
#test and make sure things make sense
key_char = ["Jon S.", "Robb S.", "Sansa S.", "OC"]
for char in key_char:
    print bookdf[char].sum() + tvdf[char].sum() == gotdf[char].sum()

True
True
True
True


In [29]:
output = []
for character in got_char_list:
    for name in got_char_list:
        if character is not name:
            stories = gotdf[(gotdf[character] == True) & (gotdf[name] == True)].count()['id']
            m_ratings = float(gotdf[(gotdf[character] == True) & (gotdf[name] == True) & (gotdf['rated'] == "M")].count()['id'])
            if stories > 0:
                m_ratio = m_ratings/stories
            else:
                m_ratio = 0
            output.append({"Character_A":character, "Character_B":name, "count":stories, "rating":m_ratio})

In [30]:
output.sort(key=lambda x:(x['count'], x['rating']))
output.reverse()

In [32]:
print output[:40]

[{'count': 959, 'Character_B': 'Sandor C.', 'Character_A': 'Sansa S.', 'rating': 0.4744525547445255}, {'count': 959, 'Character_B': 'Sansa S.', 'Character_A': 'Sandor C.', 'rating': 0.4744525547445255}, {'count': 913, 'Character_B': 'Arya S.', 'Character_A': 'Gendry W.', 'rating': 0.3833515881708653}, {'count': 913, 'Character_B': 'Gendry W.', 'Character_A': 'Arya S.', 'rating': 0.3833515881708653}, {'count': 535, 'Character_B': 'Jon S.', 'Character_A': 'Sansa S.', 'rating': 0.36074766355140186}, {'count': 535, 'Character_B': 'Sansa S.', 'Character_A': 'Jon S.', 'rating': 0.36074766355140186}, {'count': 461, 'Character_B': 'Jon S.', 'Character_A': 'Robb S.', 'rating': 0.44685466377440347}, {'count': 461, 'Character_B': 'Robb S.', 'Character_A': 'Jon S.', 'rating': 0.44685466377440347}, {'count': 412, 'Character_B': 'Arya S.', 'Character_A': 'Jon S.', 'rating': 0.41504854368932037}, {'count': 412, 'Character_B': 'Jon S.', 'Character_A': 'Arya S.', 'rating': 0.41504854368932037}, {'count

In [35]:
ft  = open("../data/got_vis_data.json", "w")
ft.write(simplejson.dumps(output[:40]))

In [34]:
simplejson.dumps(output[:40])

'[{"count": 959, "Character_B": "Sandor C.", "Character_A": "Sansa S.", "rating": 0.4744525547445255}, {"count": 959, "Character_B": "Sansa S.", "Character_A": "Sandor C.", "rating": 0.4744525547445255}, {"count": 913, "Character_B": "Arya S.", "Character_A": "Gendry W.", "rating": 0.3833515881708653}, {"count": 913, "Character_B": "Gendry W.", "Character_A": "Arya S.", "rating": 0.3833515881708653}, {"count": 535, "Character_B": "Jon S.", "Character_A": "Sansa S.", "rating": 0.36074766355140186}, {"count": 535, "Character_B": "Sansa S.", "Character_A": "Jon S.", "rating": 0.36074766355140186}, {"count": 461, "Character_B": "Jon S.", "Character_A": "Robb S.", "rating": 0.44685466377440347}, {"count": 461, "Character_B": "Robb S.", "Character_A": "Jon S.", "rating": 0.44685466377440347}, {"count": 412, "Character_B": "Arya S.", "Character_A": "Jon S.", "rating": 0.41504854368932037}, {"count": 412, "Character_B": "Jon S.", "Character_A": "Arya S.", "rating": 0.41504854368932037}, {"coun

In [ ]:
#list of popular characters
char_count = {}
for character in got_char_list:
    char_count[character] = gotdf[character].sum()
sorted_list = reversed(sorted(char_count, key=char_count.get))
for character in sorted_list:
        print "{} : {}".format(character, char_count.get(character))